In [15]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [7]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

In [8]:
table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

canada_df = pd.DataFrame(table_contents)

canada_df['Borough'] = canada_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A', 
                                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business', 
                                                     'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto', 
                                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


canada_df.head(10)

,Borough,Neighborhood,Postal Code
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Regent Park, Harbourfront",M5A
3,North York,"Lawrence Manor, Lawrence Heights",M6A
4,Queen's Park,Ontario Provincial Government,M7A
5,Etobicoke,Islington Avenue,M9A
6,Scarborough,"Malvern, Rouge",M1B
7,North York,Don Mills North,M3B
8,East York,"Parkview Hill, Woodbine Gardens",M4B
9,Downtown Toronto,"Garden District, Ryerson",M5B


In [9]:

print('Number of rows in DataFrame: {}'.format(canada_df.shape[0]))

Number of rows in DataFrame: 103


In [14]:

#geospatial_data = pd.read_csv('Geospatial_Coordinates.csv')


#canada_df = canada_df.join(geospatial_data.set_index('Postal Code'), on='Postal Code')
canada_df.head(5)

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


In [16]:
# Create a Nominatim object for geolocation
geolocator = Nominatim(user_agent="ny_explorer")

# GEt latitue and longitude values for Toronto
location = geolocator.geocode('Toronto')
tor_lat = location.latitude
tor_lon = location.longitude

# Display latitude and longitude values for Toronto
print('The geograpical coordinate of Toronto are {}, {}.'.format(tor_lat, tor_lon))

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Toronto&format=json&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x000002039E8F2940>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))